# Convert communities to jsonl so twarc can convert them back to a edgelist

In [4]:
import pandas as pd
import networkx as nx
import os, json
!pip install twarc
from twarc.expansions import ensure_flattened
import subprocess
done = True
if not done:
  jsonls = []
  for file in os.listdir('./'):
      if file.endswith('.csv'):
          df = pd.read_csv(file) 
          valid_ids = set(df["id"])

          with open("ukraine_stream_big.jsonl", "r") as infile: 
              with open(f"{file}.jsonl", "w") as outfile: 
                  jsonls.append(f"{file}.jsonl")
                  for line in infile: 
                      for tweet in ensure_flattened(json.loads(line)): 
                          if int(tweet["id"]) in valid_ids: 
                              outfile.write(json.dumps(tweet)+"\n")


  for file in jsonls:
      csv_users = file[:-6]+'_users_' + '.csv'
      subprocess.call(['twarc2', 'network', file, '--format', 'csv', csv_users, '--min-component-size', '5'])
      csv_tweets = file[:-6]+'_tweets_'+'.csv'
      subprocess.call(['twarc2', 'network', file, '--format', 'csv', csv_tweets, '--min-component-size', '5', '--nodes', 'tweets'])
      csv_tags = file[:-6]+'_tags_'+'.csv'
      subprocess.call(['twarc2', 'network', file, '--format', 'csv', csv_tags, '--min-component-size', '5', '--nodes', 'hashtags'])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 5.5 MB/s 
     |████████████████████████████████| 106 kB 29.5 MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=da2d647e154d180a98acad1d6304fceb94b9dce1f3e2fb68b91a62fc6dd2b155
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built configobj
  Attempting uninstall: humanize
    Found existing installation: humanize 0.5.1
    Uninstalling humanize-0.5.1:
      Successfully uninstalled humanize-0.5.1


In [ ]:
for file in os.listdir('./'):
    if '_users_' in file:
        print(file)
        df = pd.read_csv(file, names=['source', 'target', 'weight', 'retweet', 'reply', 'quote', 'mention'])
        df['weight'] = df['weight'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
        df['retweet'] = df['retweet'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
        df['reply'] = df['reply'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
        df['quote'] = df['quote'].map(lambda y: [int(x) for x in y.split() if x.isdigit()][0])
        graph = nx.from_pandas_edgelist(df)
        df.to_csv(file)
        nx.write_gexf(graph, f"{file}_users_.gexf")
    if '_tweets_' in file:
        print(file)
        df = pd.read_csv(file)
        df['tweet'] = df['tweet'].map(lambda y: eval(y)['type'])
        print(df.head())
        graph = nx.from_pandas_edgelist(df)
        df.to_csv(file)
        nx.write_gexf(graph, f"{file}_tweets_.gexf")
    if '_tags_' in file:
        print(file)
        df = pd.read_csv(file)
        df['weight'] = df['weight'].map(lambda y: eval(y)['weight'])
        print(df.head())
        graph = nx.from_pandas_edgelist(df)
        df.to_csv(file)
        nx.write_gexf(graph, f"{file}.gexf")
        

community111.csv_tags_.csv
   Unnamed: 0                     source      target  weight
0           0                     #Putin   #Zelenski     288
1           1                     #Putin    #Ucrania      20
2           2                     #Putin   #Zelenzsy      20
3           3                     #Putin  #Occidente       1
4           4  #BedroomSessionsRadioShow      #Rusia       6
community111.csv_tweets_.csv
   Unnamed: 0        source        target    tweet
0           0  1.573014e+18  1.572984e+18    reply
1           1  1.573014e+18  1.572612e+18  retweet
2           2  1.573015e+18  1.572654e+18  retweet
3           3  1.573015e+18  1.573004e+18  retweet
4           4  1.573015e+18  1.572925e+18  retweet
community111.csv_users_.csv
community51.csv_tags_.csv
   Unnamed: 0    source              target  weight
0           0  #Ukraine             #Tigray    1409
1           1  #Ukraine           #Ethiopia      93
2           2  #Ukraine          #Palestine       2
3         

# information cascades


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#!pip install ndlib
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep

edgelist = pd.read_csv('/content/drive/MyDrive/stream_big_edgelist.csv', names = ['index', 'source', 'target', 'weight', 'retweet', 'reply', 'quote', 'mention'])

edgelist = edgelist.iloc[1:]
print(edgelist.head())
graph = nx.from_pandas_edgelist(edgelist)
model = ep.IndependentCascadesModel(graph)
config = mc.Configuration()
config.add_model_parameter('fraction_infected', 0.1)
threshold = 0.1
for edge in graph.edges():
    config.add_edge_configuration("threshold", edge, threshold)

model.set_initial_status(config)
iterations = model.iteration_bunch(200)
print(iterations)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


   index         source           target weight retweet reply quote  \
1    0.0  antti_ellonen       EndGameWW3      1       1     0     0   
2    1.0     EndGameWW3     CarlaBabbVOA      1       1     0     0   
3    2.0     EndGameWW3     laraseligman      1       1     0     0   
4    3.0     EndGameWW3  PentagonPresSec      1       0     0     0   
5    4.0     EndGameWW3        Liveuamap      1       1     0     0   

          mention  
1   'mention': 0}  
2   'mention': 0}  
3   'mention': 0}  
4   'mention': 1}  
5   'mention': 0}  
